In [ ]:
using Catalyst

# NOTE: both models MUST preserve the same ordering of reactions in order to detect 
# how the nonlinear reactions are to be transformed using LMA

rn_nonlinear = @reaction_network begin
    @parameters σ_b σ_u ρ_b ρ_u
    σ_b, g + p → 0
    σ_u*(1-g), 0 ⇒ g + p
    ρ_u, g → g + p
    ρ_b*(1-g), 0 ⇒ p
    1, p → 0
end

rn_linear = @reaction_network begin
    @parameters σ_b_LMA σ_u ρ_b ρ_u
    σ_b_LMA, g → 0
    σ_u*(1-g), 0 ⇒ g
    ρ_u, g → g+p
    (ρ_b*(1-g)), 0 ⇒ p
    1, p → 0
end

In [ ]:
using MomentClosure

# NOTE: we have to provide the indices of binary variables in the system 
# as they are ordered in the *nonlinear* GRN.
# The distinction here between linear and nonlinear GRNs is important as in some cases 
# the internal ordering of variables of the two Catalyst models can differ
t = default_t()
@species g(t)
binary_vars = [speciesmap(rn_nonlinear)[g]]

LMA_eqs, effective_params = linear_mapping_approximation(rn_nonlinear, rn_linear, binary_vars, combinatoric_ratelaws=false)
display(effective_params)

In [ ]:
using Latexify
latexify(LMA_eqs)

In [ ]:
println(latexify(LMA_eqs))

In [ ]:
using OrdinaryDiffEq, Sundials, Plots

u0map = [:g => 1.0, :p => 0.001]
pmap = Dict(:σ_b => 0.004, :σ_u => 0.25, :ρ_b => 25.0, :ρ_u => 60.0)
tspan = (0., 15.)
dt = 0.1

oprob_LMA = ODEProblem(LMA_eqs, u0map, tspan, pmap)
sol_LMA = solve(oprob_LMA, CVODE_BDF(), saveat=dt)

plot(sol_LMA, idxs=[2], label="LMA", ylabel="⟨p⟩", xlabel="time", fmt="svg", guidefontsize=12)

In [ ]:
#savefig("../docs/src/assets/LMA_feedback_loop_mean_protein_number.svg")

In [ ]:
using FiniteStateProjection

fsp_sys = FSPSystem(rn_nonlinear, combinatoric_ratelaw=false)
# Truncate the state space of the system
# The gene has two states (G or G*) whereas we consider protein number from 0 to 100
state_space = [2, 201]

# The initial condition is the matrix of probabilities representing the state of the system
# We assume zero protein and the gene to be in the state G, hence the probability of this 
# specific state should be set to 1 initially
u0 = zeros(state_space...)
u0[2, 1] = 1.0

# construct an ODE problem from the FSPSystem and solve it
fsp_prob = ODEProblem(fsp_sys, u0, tspan, pmap)
sol_FSP = solve(fsp_prob, CVODE_BDF(), saveat=dt)

# extract the 1st order raw moments from the FSP solution 
μ_FSP = get_moments_FSP(sol_FSP, 1, "raw")
plot!(sol_FSP.t, μ_FSP[(0,1)], label="FSP", legend=:bottomright)

In [ ]:
#savefig("../docs/src/assets/LMA+FSP_feedback_loop_mean_protein_number.svg")

In [ ]:
using TaylorSeries, HypergeometricFunctions

function t_pFq(α::AbstractVector, β::AbstractVector, a::Taylor1)
    order = a.order
    aux = pFq(α, β, constant_term(a))
    c = Taylor1(aux, order)

    iszero(order) && return c

    coeffs = t_pFq(α.+1, β.+1, Taylor1(a[0:end-1], a.order-1))
    factor = prod(α)/prod(β)
    for k in 1:length(a)-1
        c[k] = sum(i * a[i] * coeffs[k-i] for i in 1:k) * factor / k
    end

    return c

end

In [ ]:
# calculate the raw moments up to time t at a fine temporal resolution
T = 15.0
tspan = (0., T)
dt = 0.001
oprob_LMA = remake(oprob_LMA; tspan)
sol_LMA = solve(oprob_LMA, CVODE_BDF(), saveat=dt)

# rebuild the symbolic expression for the effective parameter as a function of raw moments
using ModelingToolkit: get_ps, getname
ps = get_ps(rn_nonlinear)
symbol_to_symbolic = Dict(Pair.(getname.(ps), ps))
p_sub = [symbol_to_symbolic[p[1]] => p[2] for p in pmap]
μ_sym = unknowns(LMA_eqs.odes)

avg_σ_b_sym = first(values(effective_params))
fn = build_function(substitute(avg_σ_b_sym, p_sub), μ_sym)
avg_σ_b = eval(fn)
# evaluate the time-averaged value of the effective parameter
@time σ_b_avg = sum(avg_σ_b.(sol_LMA[:])) * dt / T

In [ ]:
# need higher-precision numerics as Float64 can be unstable here due to very small numbers
# DoubleFloats is sufficient for this example and much more efficient than BigFloat
using DoubleFloats

# define the numerical values of the parameters
σ_u = pmap[:σ_u]; ρ_b = pmap[:ρ_b]; ρ_u = pmap[:ρ_u]
Σ = 1 + σ_b_avg + σ_u
ρ_Δ = ρ_b - ρ_u

n = 100 # expansion order (or max protein number to evaluate)
w₀ = -1 # value around which to expand

# compute the Taylor expansion (note the use of Double64)
w = w₀ + Taylor1(Double64, n)
@time f = σ_b_avg/(Σ-1)*exp(-T*(Σ-1))*exp(-ρ_u*w*exp(-T))*t_pFq([σ_u], [Σ], -ρ_Δ*w*exp(-T))
@time g = σ_u/(Σ-1)*exp(-ρ_u*w*exp(-T))*t_pFq([-σ_b_avg], [2-Σ], -ρ_Δ*w*exp(-T))

@time G00 = exp(ρ_b*w)*(f * t_pFq([1-σ_b_avg], [2-Σ], -ρ_Δ*w) +
                  g * t_pFq([1+σ_u], [Σ], -ρ_Δ*w) )

@time G11 = σ_u^(-1) * exp(ρ_b*w) * (-σ_u*f*t_pFq([-σ_b_avg], [2-Σ], -ρ_Δ*w) +
                                σ_b_avg*g*t_pFq([σ_u], [Σ], -ρ_Δ*w))

probs = (G00+G11).coeffs

# check that the probability distribution is more or less normalised to 1
# need higher numerical precision if not
isapprox(sum(probs), 1.0, rtol=1e-2)

In [ ]:
plot(0:n, probs, xlabel="n", ylabel="P(n, t=4)", label="LMA", fmt="svg")
# plot the FSP probability of protein number by marginalising over the gene states
plot!(0:n, sum(sol_FSP[:, 151], dims=1)'[1:n+1], label="FSP")

In [ ]:
#savefig("../docs/src/assets/LMA+FSP_feedback_loop_distribution.svg")